# Client


In [1]:
from eventbus.event_net import Bridge

import asyncio
import aiohttp
from eventbus.event_net.transport import AioTransport
from eventbus import bus
from plugins.core.log import Log

ws_url = "http://192.168.10.200:5000/ws"
ws_url = "https://roulotte.leaf49.org/ws"

CLIENT_ID = "client123"

cookies = {
    "leaf-token": CLIENT_ID,
    "CF-Access-Client-Secret": "ddd5c9040aed50768438b659cf53f11828c2e27f1c97e1a5b5956f0dfe8eba4f",
    "CF-Access-Client-Id": "e754ee18b1f1ad768a26a3e53f2e68cd.access",
}

bus.LEAF_ID = CLIENT_ID

Log()


async def sender_task():
    for i in range(3):
        event = {"topic": "?echo", "src": bus.LEAF_ID, "dst": "server", "data": f"Really? {i}"}
        print("SEND", event)
        await bus.emit(event)
        await asyncio.sleep(4)
    await bus.emit({"topic": "!bye", "src": bus.LEAF_ID, "dst": "server"})


async with aiohttp.ClientSession(cookies=cookies) as session:
    # connect to websocket
    async with session.ws_connect(ws_url) as ws:
        bridge = Bridge("sender", AioTransport(ws))  # type: ignore
        bridge.dst_filter = {"server"}
        bridge.receive_timeout = 1
        asyncio.create_task(sender_task())
        await bridge.run()

print("DONE")

WSServerHandshakeError: 200, message='Invalid response status', url='https://leaf49.cloudflareaccess.com/cdn-cgi/access/login/roulotte.leaf49.org?kid=21208b2f30f8d03382bb646936cb837a0b5e8cb36e48beb8c2846dd1b65fb25c&redirect_url=/ws&meta=eyJraWQiOiI5MjFjNzNkNmQ3Y2QwYTFiYTA1Mjk0NWM0YzgxNDY2ZTMyZWJjYjI1ZTYwN2Y5OTliNDljMTZiOGEwYmE4ZTlkIiwiYWxnIjoiUlMyNTYiLCJ0eXAiOiJKV1QifQ.eyJzZXJ2aWNlX3Rva2VuX3N0YXR1cyI6ZmFsc2UsImlhdCI6MTcyNTMwMTg3Miwic2VydmljZV90b2tlbl9pZCI6IiIsImF1ZCI6IjIxMjA4YjJmMzBmOGQwMzM4MmJiNjQ2OTM2Y2I4MzdhMGI1ZThjYjM2ZTQ4YmViOGMyODQ2ZGQxYjY1ZmIyNWMiLCJob3N0bmFtZSI6InJvdWxvdHRlLmxlYWY0OS5vcmciLCJhcHBfc2Vzc2lvbl9oYXNoIjoiMGI3MTM1ZjFhYmMwNWFjMWJmYzRjNDNjNjU0YWM2YmI1ZGYwNzNmMjFiMmVhODQ4NmFiMTM5Nzk5MzRkMDQ1MCIsIm5iZiI6MTcyNTMwMTg3MiwiaXNfd2FycCI6ZmFsc2UsImlzX2dhdGV3YXkiOmZhbHNlLCJ0eXBlIjoibWV0YSIsInJlZGlyZWN0X3VybCI6Ilwvd3MiLCJtdGxzX2F1dGgiOnsiY2VydF9pc3N1ZXJfc2tpIjoiIiwiY2VydF9wcmVzZW50ZWQiOmZhbHNlLCJjZXJ0X3NlcmlhbCI6IiIsImNlcnRfaXNzdWVyX2RuIjoiIiwiYXV0aF9zdGF0dXMiOiJOT05FIn0sImF1dGhf